# Segmenting and Clustering Neighborhoods in Toronto

In [1]:
# Initialize libraries
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup as bs

The next section will open the HTML file with BeautifulSoup and confirm successful input. The HTML file was downloaded into a local folder.

In [2]:
# Open html file with BeautifulSoup and confirm the name of the tree
with open('List of postal codes of Canada_ M - Wikipedia.htm') as fp:
    soup = bs(fp)
soup.name

'[document]'

Then we will parse for a table and create headers for our dataframe. After that, we will append each row from the table to the dataframe, making sure to ignore rows with no assigned Borough, and setting the Neighborhood to the Borough if no Neighborhood is present.

In [3]:
# Check the table elements and create a dataframe

# First check table headers and create dataframe columns
labels = []
len_col = 0
for x in soup.table.find_all('th'):
    y = str(x)
    #for x in headers:
    y = y.strip('\<th\>')
    y = y.strip('</')
    y = y.strip('\n')
    labels.append(y)
    len_col = len_col + 1
t_df = pd.DataFrame(columns=labels)

# Next check each table row and append them to the dataframe
a = 0
z = {}
counter = 0
for x in soup.table.find_all(['td']):
    if a == len_col:
        a = 0
        if z['Neighbourhood'].strip('\n') == 'Not assigned':
            z['Neighbourhood'] = z['Borough']
        if z['Borough'] != 'Not assigned':
            t_df = t_df.append(z, ignore_index=True)
        z = {}
        counter = counter + 1
    temp = x.string
    if temp == None:
        temp = x.find('a').string
        temp = temp.strip('\n')
    z[labels[a]]=temp
    a = a + 1

t_df[0:10]


,Postcode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M5A,Downtown Toronto,Regent Park
4,M6A,North York,Lawrence Heights
5,M6A,North York,Lawrence Manor
6,M7A,Queen's Park,Queen's Park
7,M9A,Etobicoke,Islington Avenue
8,M1B,Scarborough,Rouge
9,M1B,Scarborough,Malvern


Next we will consolidate rows with the same Postcode by appending their Neighborhood values.

In [4]:
# Clean up duplicate postcodes, first by counting the number of duplicates, then appending the Neighborhood to the first entry

for x in t_df['Postcode']:
    dupe_list = t_df.loc[t_df['Postcode']==x].index
    dup = (len(t_df.loc[t_df['Postcode']==x].index))

    while dup > 1:
        if t_df.iloc[dupe_list[0]][2].find(t_df.iloc[dupe_list[dup-1]][2].strip('\n')) < 0:

            t_df.iloc[dupe_list[0]][2] = t_df.iloc[dupe_list[0]][2].strip('\n') + ', ' + t_df.iloc[dupe_list[dup-1]][2].strip('\n')
        dup = dup - 1
t_df.drop_duplicates(subset=['Postcode'], keep='first', inplace=True)        
t_df[0:20]

,Postcode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Harbourfront, Regent Park"
4,M6A,North York,"Lawrence Heights, Lawrence Manor"
6,M7A,Queen's Park,Queen's Park
7,M9A,Etobicoke,Islington Avenue
8,M1B,Scarborough,"Rouge, Malvern"
10,M3B,North York,Don Mills North
11,M4B,East York,"Woodbine Gardens, Parkview Hill"
13,M5B,Downtown Toronto,"Ryerson, Garden District"


Finally we confirm the shape of the dataframe.

In [5]:
t_df.shape

(103, 3)